# How to connect to Azure OpenAI service via OpenAI Python SDK


First you need to get or create an Azure OpenAI resource. 
You can do it here: https://portal.azure.com/#view/Microsoft_Azure_ProjectOxford/CognitiveServicesHub/~/OpenAI

Click on the chosen resource and than the "Develop" button in the middle of the screen.

Copy the API key and endpoint and paste it into `.env` file in the `examples` folder. 

import the OpenAI Python SDK and load your API key from the environment

In [2]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

#load_dotenv()


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

deployment_name = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_version = config["AZURE_OPENAI_API_VERSION"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-5:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}..."
)
print(f"Azure OpenAI deployment name={deployment_name[0:7]}...")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...d67b3
Azure OpenAI api endpoint=https://openai-pr...
Azure OpenAI deployment name=gpt-35T...
Azure OpenAI api version=2023-07-01-preview


In [3]:
aoai_client = AzureOpenAI(
    api_key=azure_oai_key,
    azure_endpoint=api_base_url,
    api_version=api_version,
)

In [4]:

models = aoai_client.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id   
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
pprint(models)



[Model(id='gpt-35-turbo', created=None, object='model', owned_by=None, capabilities={'fine_tune': False, 'inference': True, 'completion': True, 'chat_completion': True, 'embeddings': False}, lifecycle_status='generally-available', deprecation={'inference': 1720137600}, status='succeeded', created_at=1678320000, updated_at=1688601600),
 Model(id='gpt-35-turbo-0301', created=None, object='model', owned_by=None, capabilities={'fine_tune': False, 'inference': True, 'completion': True, 'chat_completion': True, 'embeddings': False}, lifecycle_status='generally-available', deprecation={'inference': 1720137600}, status='succeeded', created_at=1678320000, updated_at=1688601600),
 Model(id='gpt-35-turbo-0613', created=None, object='model', owned_by=None, capabilities={'fine_tune': False, 'inference': True, 'completion': False, 'chat_completion': True, 'embeddings': False}, lifecycle_status='generally-available', deprecation={'fine_tune': 1718236800, 'inference': 1718236800}, status='succeeded', 

In [5]:
response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "system",
            "content": "You are teacher and help studends learn phisics, geography and chemistry.",
        },
        {
            "role": "user",
            "content": "Name the planets of the solar system.",
        },
    ],
)

pprint(response.choices[0].message.content)

('The planets of the solar system, in order from the Sun, are:\n'
 '\n'
 '1. Mercury\n'
 '2. Venus\n'
 '3. Earth\n'
 '4. Mars\n'
 '5. Jupiter\n'
 '6. Saturn\n'
 '7. Uranus\n'
 '8. Neptune\n'
 '\n'
 'Please note that Pluto was considered a planet until 2006 when it was '
 'reclassified as a dwarf planet.')


## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://localhost/

Create new project with you `project_slug`or edit existing one.

Add your own Azure OpenAI provider by editing the project settings, this will map the Azure OpenAI endpoint to new proxy prompt sail URL. 

Set the `api base url` to your Azure OpenAI endpoint like
 
'https://**azure_openai_resource**.openai.azure.com/'
 
 and add meaningfull `deployment name`.


In mongo it will create new entry in `ai_providers` array, similar to this one

```bash
{
     ai_providers: [
        {
            deployment_name: 'azure US deployment',
            slug: 'azure-us-deployment',
            api_base: 'https://[azure_openai_resource].openai.azure.com/',
            description: '',
            provider_name: 'Azure OpenAI'
        }
    ],
}
```

In this case we will use the default `project 2` settings:
* with project_slug -> 'project2' 
* deployment_name -> 'azure-us-deployment'
resulting in promptsail proxy url like this: 

**"http://localhost:8000/project2/azure-us-deployment"**

In [6]:

ps_api_base = "http://localhost:8000/project2/azure-us-deployment"

ps_api_base = "http://localhost:8000/edu-project/ps-us2"


ps_client = AzureOpenAI(
    api_key=azure_oai_key,
    azure_endpoint=ps_api_base,
    api_version=api_version,
)


response = ps_client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "system",
            "content": "You are teacher and help studends learn phisics, geography and chemistry.",
        },
        {
            "role": "user",
            "content": "Name the plantets in solar system.",
        },
    ],
)

pprint(response.choices[0].message.content)



('The planets in our solar system, in order from closest to the Sun to '
 'farthest, are:\n'
 '\n'
 '1. Mercury\n'
 '2. Venus\n'
 '3. Earth\n'
 '4. Mars\n'
 '5. Jupiter\n'
 '6. Saturn\n'
 '7. Uranus\n'
 '8. Neptune\n'
 '\n'
 'Please note that Pluto used to be considered the ninth planet, but it has '
 'since been reclassified as a "dwarf planet" by the International '
 'Astronomical Union (IAU).')
